In [1]:
import datetime
import calendar
import pandas as pd
import os.path
import numpy as np

import time
import requests
currencies = ['BTC', 'ETH', 'LSK', 'GAME', 'BCC', 'LTC', 'DASH']
relays = ['PLN', 'EUR', 'USD', 'BTC']
period = 1.2

def check_bid_ask():
    diff_f = 1e-2
    diff_c = 1e-8

    pairs = {}

    for relay in relays:
        for c in currencies:
            if c != relay:
                url = 'https://bitbay.net/API/Public/' + c + relay + '/all.json'
                time.sleep(period)
                r = requests.get(url)
                r = r.json()
#                 print(r)
                if relay == 'BTC':
                    my_bid = round(r['bid']+diff_c,8)
                    my_ask = round(r['ask']-diff_c,8)
                else:
                    my_bid = round(r['bid']+diff_f,2)
                    my_ask = round(r['ask']-diff_f,2)
                pairs[c+relay] = {'bid': r['bid'], 'ask': r['ask'], 'my_bid': my_bid, 'my_ask': my_ask}
#                 print(pairs[c1+c2])
    return pairs

# pairs = check_bid_ask()


In [ ]:
def transform_to_BTC_value(pairs):
    offerts_BTC = {}

    for c in currencies:
        offerts_bid = {}
        offerts_ask = {}
        for relay in ['PLN', 'EUR', 'USD']:
            offerts_bid['bid_'+relay] = pairs[c+relay]['my_bid']/pairs['BTC'+relay]['my_ask']
            offerts_ask['ask_'+relay] = pairs[c+relay]['my_ask']/pairs['BTC'+relay]['my_bid']
        if c != 'BTC':
            offerts_bid['bid_BTC'] = pairs[c+'BTC']['my_bid']
            offerts_ask['ask_BTC'] = pairs[c+'BTC']['my_ask']
        offerts_BTC[c] = {'bid': offerts_bid, 'ask': offerts_ask}
    
    return offerts_BTC 

# offerts_BTC = transform_to_BTC_value(pairs)


def plot_currency_BTC_value(offerts_BTC):
    matplotlib.rcParams['figure.figsize'] = (18,4)
    for key in offerts_BTC.keys():
        a = {"x":[], "y":[], "label":[]}
        for label, coord in offerts_BTC[key]['ask'].items():
            a["x"].append(coord)
            a["y"].append(0)
            a["label"].append(label)

        b = {"x":[], "y":[], "label":[]}
        for label, coord in offerts_BTC[key]['bid'].items():
            b["x"].append(coord)
            b["y"].append(0)
            b["label"].append(label)

        # display scatter plot data
        plt.title(key, fontsize=20)
        plt.xlim([min(b["x"]),max(a["x"])])    
        plt.scatter(a["x"], a["y"], marker = 'o', color = 'b', s = 100)
        plt.scatter(b["x"], b["y"], marker = 'o', color = 'r', s = 100)

        for label, x in zip(a["label"], a["x"]):
            plt.annotate(label, xy = (x, 0))

        for label, x in zip(b["label"], b["x"]):
            plt.annotate(label, xy = (x, 0))

        plt.show()
        
# plot_currency_BTC_value(offerts_BTC)

In [ ]:
def find_profitable_links(offerts_BTC):
    fee = 0.0037
    threshold = 0.05
    gap = []
    for c in currencies:
        for relay in relays:
            if(c != relay):
                g = offerts_BTC[c]['ask']['ask_'+relay]/offerts_BTC[c]['bid']['bid_'+relay]*(1-2*fee)-1
                gap.append([c+relay,c,relay,g])

    gap_sorted = sorted(gap, key=lambda x : -x[3])
    gap = [x for x in gap_sorted if x[3] > threshold]
    return gap
   
# gap = find_profitable_links(offerts_BTC)
# # print(len(gap_sorted))
# print(gap)
# print(len(gap))

# for c in currencies:
#     print(c, len([x for x in gap if x[1] == c]))
    
# for relay in relays:
#     print(relay, len([x for x in gap if x[2] == relay]))
 

In [2]:
import requests
import urllib
import hmac
import hashlib
import json

# import ntplib
# from time import ctime
# c = ntplib.NTPClient()
# t = c.request('pool.ntp.org').tx_time
# print(t)
# print(int(time.time()))

def info():
    time.sleep(period)
    q = {'method': 'info'}
    return request(q)

def info():
    time.sleep(period)
    q = {'method': 'info'}
    return request(q)

def orderbook(order_currency, payment_currency):
    time.sleep(period)
    q = {'method': 'orderbook', 'order_currency': order_currency, 'payment_currency': payment_currency}
    return request(q)

def bid(currency, amount, payment_currency, rate):
    time.sleep(period)
    q = {'method': 'trade', 'type': 'bid', 'currency': currency , 'amount': amount, 'payment_currency': payment_currency, 'rate': rate}
    """return: order_id, error"""
    return request(q)

def ask(currency, amount, payment_currency, rate):
    time.sleep(period)
    print('ask', currency, amount, payment_currency, rate)
    q = {'method': 'trade', 'type': 'ask', 'currency': currency , 'amount': amount, 'payment_currency': payment_currency, 'rate': rate}
    """return: order_id, error"""
    return request(q)

def cancel(id):
    time.sleep(period)
    q = {'method': 'cancel', 'id': id}
    """return: success, error"""
    return request(q)

def cancel_all():
    o = current_orders()
    for order in o:
        cancel(order["order_id"])

def current_orders(limit = 100):
    time.sleep(period)
    q = {'method': 'orders', 'limit': limit}
    """return: order_id, order_currency, order_date, payment_currency, type, status, units, start_units, current_price, start_price"""
    print(request(q))
    return request(q)

def transactions(currency1, currency2):
    time.sleep(period)
    q = {'method': 'transactions', 'market': currency1+'-'+currency2}
    """return: date, type, market, amount, rate, price"""
    return request(q)

def print_json(js):
    print(json.dumps(js, indent=2))
    
def request(query):
    
    
    API_key = b'cef234b2-7a8b-4621-94b4-228c45bfa072'
    API_secret = b'f1699871-d42e-476d-a86e-ac65fdf0bf89'
    url = 'https://bitbay.net/API/Trading/tradingApi.php'
    
    for attempt in range(5):
        try:
            query['moment'] = int(time.time())
            post = urllib.parse.urlencode(query)
            API_Hash = hmac.new(API_secret, post.encode('utf-8'), hashlib.sha512)
            headers = {'API-Key': API_key, 'API-Hash': API_Hash.hexdigest()}
    
            answer = requests.post(url, data=post, headers=headers)
#             print(answer)
#             print(answer.json())
            if 'code' in answer.json():
                print('code in answer')
                raise
        except:
            time.sleep(period)
        else:
            break
    else:
        print('Proba polaczenia nieudana')
        
    return answer.json()

In [3]:
def print_wallet(wallet):
    sum = 0
    for c in wallet['balances'].keys():
        a = float(wallet['balances'][c]['available'])
        l = float(wallet['balances'][c]['locked'])
        s = round(a + l, 8)

        try:
            print(c, a, l, s, round(s*offerts_BTC[c]['ask']['ask_BTC']*1000, 1))
            sum += round(s*offerts_BTC[c]['ask']['ask_BTC'], 8)
        except:
            print(c, a, l, s, s)
            
def calculate_wallet(wallet):
    minimum = {}
    total_BTC = 0
    extended_wallet = {}
    for c in currencies:
        a = float(wallet['balances'][c]['available'])
        l = float(wallet['balances'][c]['locked'])
        s = round(a + l, 8)

        extended_wallet[c] = {'available': a, 'locked': l, 'sum': s}
            
    for r in relays:
        if r != 'BTC':
            a = float(wallet['balances'][r]['available'])
            l = float(wallet['balances'][r]['locked'])
            s = round(a + l, 2)
            extended_wallet[r] = {'available': a, 'locked': l, 'sum': s}
    
    return extended_wallet

In [ ]:
def sell_currency(currency, f='ask'):
    print('-------------')
    print(currency)
    print()
    links = len([x for x in gap if x[1] == currency])
    print('links', links)
    
    if links > 0:
        for relay in relays:
            if len([x for x in gap if x[0] == currency+relay]) == 1:
                deposit = float(wallet['balances'][currency]['available'])+float(wallet['balances'][currency]['locked'])
                amount = round(deposit/links/4,8)

                if currency+relay+f in orders:
                    print(orders[currency+relay+f]['order_id'])
                    if len([item for item in o if item["order_id"] == orders[currency+relay+f]['order_id']]) == 1:                   
                        if pairs[currency+relay][f] != orders[currency+relay+f]['rate']:
                            cancel(orders[currency+relay+f]['order_id'])
                            del orders[currency+relay+f]
                            print('Ktoś dał lepszą ofertę')
                        else:
                            print("Oferta istnieje")
                    else:
                        del orders[currency+relay+'ask']
                        print('Prawdopodobnie udana transakcja')
                else:
                    if amount > 0.0001:
                        t = ask(currency,amount,relay,pairs[currency+relay]['my_'+f])
                        orders[currency+relay+f] = t
                        print(t)
                        print(time.time(), "Oferta wystawiona ", orders[currency+relay+f]['order_id'])
                    else:
                        print(time.time(), "Brak wolnych środków")
            else:
                if currency+relay+f in orders:
                    print(cancel(orders[currency+relay+f]['order_id']))
                    del orders[currency+relay+f]
                print("Niezyskowne")
                
def buy_currency(relay, f='bid'):
    print('-------------')
    print(relay)
    print()
    links = len([x for x in gap if x[2] == relay])
    
    if links > 0:
        for currency in currencies:
            if len([x for x in gap if x[0] == currency+relay]) == 1:  
                deposit = float(wallet['balances'][currency]['available'])+float(wallet['balances'][currency]['locked'])
                amount = deposit/links             

                if currency+relay+f in orders:
                    if len([item for item in o if item["order_id"] == orders[currency+relay+f]['order_id']]) == 1:                        
                        if pairs[currency+relay][f] != orders[currency+relay+f]['rate']:
                            cancel(orders[currency+relay+f]['order_id'])
                            del orders[currency+relay+f]
                            print('Ktoś dał lepszą ofertę')
                        else:
                            print("Oferta istnieje")
                    else:
                        del orders[currency+relay+f]
                        print('Prawdopodobnie udana transakcja')
                else:
                    if relay == 'BTC':
                        thres = 0.0001
                        amount = amount/8
                    else:
                        thres = 1.0
                    if amount > thres:
                        amount = round(amount/pairs[currency+relay]['my_'+f],8) 
                        t = bid(currency,amount,relay,pairs[currency+relay]['my_'+f])                            
                        orders[currency+relay+f] = t
                        print(t)
                        print(time.time(), "Oferta wystawiona ", orders[currency+relay+f]['order_id'])
                    else:
                        print(time.time(), "Brak wolnych środków")
            else:
                if currency+relay+f in orders:
                    print(cancel(orders[currency+relay+f]['order_id']))
                    del orders[currency+relay+f]
                print("Niezyskowne")

# print_json(orderbook('BTC', 'PLN'))

# print(cancel_all())
# pairs = check_bid_ask()
# print(calculate_wallet(info(), pairs))

# orders = {}
# while True:
#     pairs = check_bid_ask()
#     offerts_BTC = transform_to_BTC_value(pairs)
#     plot_currency_BTC_value(offerts_BTC)
#     gap = find_profitable_links(offerts_BTC)
#     print(len(gap))
#     wallet = info()
#     print_wallet(wallet)
#     o = current_orders()
    
#     for currency in currencies:
#         sell_currency(currency) 
#     for relay in relays:
#         buy_currency(relay)



In [4]:
import pandas as pd
import time
import os.path
import requests
from shutil import copyfile
import cProfile
import datetime
import talib

In [5]:
def resample_df(df, frequency = '1H', window1 = 6, window2 = 40):
    one_hot = pd.get_dummies(df['type'])
    df = df.join(one_hot)

    df_pv = df.resample(frequency).agg({'price': np.mean, 'amount': np.sum, 'buy': np.sum, 'sell': np.sum})
    df_ohlc = df['price'].resample(frequency).ohlc()

    df = pd.concat([df_pv, df_ohlc], axis=1)
    df = df.fillna(method='pad')
    df = df.dropna()
    
    df['SMA(24)'] = df['price'].rolling(24).mean()
    df['SMA(12)'] = df['price'].rolling(12).mean()
    
    return df


def download_trades(pair='BTCPLN', since=0):
    url = 'https://bitbay.net/API/Public/' + pair + '/trades.json?since=' + str(since)
    time.sleep(min_period)
    r = requests.get(url)
#     print(r.json())
    return r.json()

def update_trades(pair, since):
    trades_file = 'Data/trades_BitBay_' + pair + '.csv'
    
    try:
        trades = download_trades(pair=pair, since=since)
    except (KeyboardInterrupt, SystemExit):
        raise
    except:
        print(pair, "except")
        pass
    else:
        if len(trades) > 0:
            df_temp = pd.DataFrame(data=trades, columns=['date','price','type','amount','tid'])
            df_temp[['amount','price']] = df_temp[['amount','price']].astype('float64')
            df_temp.index = df_temp['tid'].values

            try:
                copyfile(trades_file, trades_file+'temp')
                df_temp.to_csv(trades_file+'temp', mode='a', header=0)
                os.replace(trades_file+'temp', trades_file)
                copyfile(trades_file, trades_file+'bac')
                since = df_temp['tid'].iloc[-1]
            except:
                print(pair, "Problem zapisu")
                copyfile(trades_file+'bac', trades_file)          

            print(pair, pd.to_datetime(df_temp['date'], unit='s').tail(1), df_temp.tail(1))
            
    return since, df_temp

In [ ]:
period = 10
min_period = 1.05
sinces = {}

cooldown = datetime.timedelta(seconds=5)
min_PLN = 1e-1
min_BTC = 1e-6

buysell = 0.000
n = 4
limit_sell = buysell
limit_buy = buysell

frequency = '5min'
currency = 'BTC'
relay = 'PLN'
pair = currency + relay
trades_file = 'Data/trades_BitBay_' + pair + '.csv'

df_trades = pd.read_csv(trades_file, index_col=0, usecols=['date','price', 'type', 'amount'], header=0, engine='python')
sinces[pair] = df_trades['tid'].iloc[-1]

while True:  
    sinces[pair], df_temp = update_trades(pair=pair, since=sinces[pair])
    df_trades = df_trades.append(df_temp, ignore_index=True)
    dataframe = df_trades
    dataframe.index = pd.to_datetime(dataframe.index, unit='s')
    trades_ohlc = resample_df(dataframe, frequency)

    wallet = calculate_wallet(info())
    PLN = wallet[relay]['sum']
    BTC = wallet[currency]['sum']

    window = trades_ohlc.iloc[-(n+1):]
    average = trades_ohlc['SMA(12)'][-1]
    print(trades_ohlc.index[-1])

    mfi = talib.MFI(window['high'].values, window['low'].values, window['close'].values, window['amount'].values, n)[-1]
    cci = talib.CCI(window['high'].values, window['low'].values, window['close'].values, n)[-1]
    cmo = talib.CMO(window['close'].values, n)[-1]

    if cci > 100 and mfi > 80 and cmo > 50:
        sell = True
    else:
        sell = False

    if cci < -100 and mfi < 20 and cmo < -50:
        buy = True
    else:
        buy = False

    buy_price = average*(1+limit_buy)        
    if(buy and BTC > min_BTC):
        print('ask', buy_price)
        cancel_all()
        wallet = calculate_wallet(info())
    #     t = ask(currency, wallet[currency]['available'], relay, round(buy_price, 2))
    #     print(time.time(), "Oferta wystawiona ASK ", t)

    sell_price = average*(1-limit_sell)
    if(sell and PLN > min_PLN):
        print('bid', sell_price)
        cancel_all()
        wallet = calculate_wallet(info())
    #     t = bid(currency, round(wallet[relay]['available']/sell_price, 8), relay, round(sell_price, 2))                            
    #     print(time.time(), "Oferta wystawiona BID ", t)
    
        